# Datarobot generative AI capabilities overview

**Author**: Gustav von Zitzewitz</br>
**Date**: May 29, 2024

In November 2023, DataRobot launched its new generative AI products including LLM playgrounds and vector databases for doing Retriever Augmented Generation (RAG) workflows.</br>
This accelerator walks through all of the new DataRobot generative AI capabilities and serves as a template for code-first users.
It showcases how to create and use playgrounds, connect vector databases and LLMs, and how to query and compare them. It uses the official DataRobot documentation as an example. Furthermore, it combines DataRobot's well-known custom model capabilities to allow users to bring their own LLMs and vector databases. It shows how to link, use, and deploy them together with the ones offered by DataRobot.

## Task overview

- Create DataRobot generative AI playgrounds
- Create DataRobot LLM-Blueprints
- Chat prompt single LLM-Blueprints
- Create DataRobot vector databases
- Investigate vector database assets
- Create LLM-Blueprints that use vector databases (RAG)
- Create and deploy external vector databases
- Create and deploy external LLMs
- Validate and register external vector databases and LLMs to playgrounds
- Comparison prompt multiple LLM-Blueprints
- Deploy LLM-Blueprints and make predictions

## Setup

### Install libraries

In [ ]:
# We start with installing the newest datarobotx package
# This is necessary to leverage sophisticated model deployment features
!pip install -U datarobotx

Next, install all external libraries that you are going to use in this notebook

In [ ]:
!pip install langchain==0.0.333 faiss-cpu==1.7.4 sentence-transformers==2.2.2 pyarrow==16.1.0

### Import libraries and modules

In [1]:
# After installing the packages we import all modules that we'll use in this notebook
import json

import datarobot as dr
from datarobot.enums import VectorDatabaseChunkingMethod, VectorDatabaseEmbeddingModel
from datarobot.models.genai.chat_prompt import ChatPrompt
from datarobot.models.genai.comparison_prompt import ComparisonPrompt
from datarobot.models.genai.custom_model_llm_validation import CustomModelLLMValidation
from datarobot.models.genai.llm import LLMDefinition
from datarobot.models.genai.llm_blueprint import LLMBlueprint, VectorDatabaseSettings
from datarobot.models.genai.playground import Playground
from datarobot.models.genai.vector_database import (
    ChunkingParameters,
    CustomModelVectorDatabaseValidation,
    VectorDatabase,
)
import datarobotx as drx
from langchain.docstore.document import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import MarkdownTextSplitter
from langchain.vectorstores.faiss import FAISS
import pandas as pd

### Connect to DataRobot

Read more about different options for [connecting to DataRobot from the Python client](https://docs.datarobot.com/en/docs/api/api-quickstart/api-qs.html).

In [ ]:
dr.Client(token="PUT_YOUR_DR_API_TOKEN_HERE")

## Create and use DataRobot generative AI playgrounds

In the following cells, you will create new or load existing use cases, playgrounds, and LLM blueprints.

In [ ]:
# GenAI works within a use case. Create a new use case.
use_case = dr.UseCase.create("DataRobot GenAI Accelerator Use Case")
use_case

UseCase(id=6645f9391bdeb4610c5f198d, name=DataRobot GenAI Accelerator Use Case, description=None, models=0, projects=0, datasets=0, notebooks=0, applications=0)

In [30]:
# Optionally, load an existing use case instead
use_case = dr.UseCase.get("6645ef9015de77fa7c5f1b9a")
use_case

UseCase(id=6645ef9015de77fa7c5f1b9a, name=DataRobot GenAI Accelerator Use Case, description=None, models=0, projects=0, datasets=0, notebooks=0, applications=0)

In [ ]:
# GenAI LLMs live within a playground. Create a new playground.
playground = Playground.create(
    name="Datarobot Documentation Playground", use_case=use_case
)
playground

Playground(id=6645f94aff64531a95c4ce46, name=Datarobot Documentation Playground)

In [3]:
# Optionally, load an existing playground instead.
playground = Playground.get("6646f44bdcf3d388f96a2779")
playground

Playground(id=6646f44bdcf3d388f96a2779, name=Playground 2024-05-17 08:08:09)

In [ ]:
# See what kind of LLMs are available. By default this is returned as a dict
# for easy readability. This includes a list of allowed settings for each LLM along with
# any constraints on the values.
llms_dict = LLMDefinition.list()
print([llm["id"] for llm in llms_dict])

['azure-openai-gpt-3.5-turbo', 'azure-openai-gpt-3.5-turbo-16k', 'azure-openai-gpt-4', 'azure-openai-gpt-4-32k', 'amazon-titan', 'anthropic-claude-2', 'google-bison']


In [ ]:
# Use GPT 3.5 for the example
llms = LLMDefinition.list(as_dict=False)
gpt = llms[0]

# To interact with GPT you need to create an LLM blueprint with the settings that you want.
# Since the allowed settings depend on the LLM type you take a generic dict with those values
# that will be validated during the creation of the blueprint. Review the allowed
# settings for GPT here.
[print(setting.to_dict()) for setting in gpt.settings]

{'id': 'system_prompt', 'name': 'System prompt', 'description': 'System prompt guides the style of the LLM response. It is a "universal" prompt, prepended to all individual prompts.', 'type': 'string', 'format': 'multiline', 'is_nullable': True, 'constraints': {'type': 'string', 'min_length': None, 'max_length': 500000, 'allowed_choices': None}, 'default_value': ''}
{'id': 'max_completion_length', 'name': 'Max completion tokens', 'description': "Maximum number of tokens allowed in the completion. The combined count of this value and prompt tokens must be below the model's maximum context size (4096), where prompt token count is comprised of system prompt, user prompt, recent chat history, and vector database citations.", 'type': 'integer', 'format': None, 'is_nullable': True, 'constraints': {'type': 'integer', 'min_value': 1, 'max_value': 4096}, 'default_value': 1024}
{'id': 'temperature', 'name': 'Temperature', 'description': 'Temperature controls the randomness of model output. Enter

In [ ]:
# Create the LLM blueprint with custom settings defining it's "personality"
llm_settings = {
    "system_prompt": (
        "You are a DataRobot engineer and know everything about the datarobot platform. "
        "If you don't know the answer just pretend to be confident and make up something that sounds reasonable."
    ),
    "max_completion_length": 1024,  # With decent completion length
    "temperature": 1.0,  # With a high degree of variability
}

llm_blueprint = LLMBlueprint.create(
    playground=playground,
    name="Overconfident DataRobot Engineer GPT 3.5",
    llm=gpt,
    llm_settings=llm_settings,
)
llm_blueprint

LLMBlueprint(id=6645f956dcf3d388f96a21f6, name=Overconfident DataRobot Engineer GPT 3.5, is_saved=False)

In [ ]:
# Now chat with the LLM blueprint
prompt = ChatPrompt.create(
    "How to deploy a custom model in datarobot?",
    llm_blueprint,
    wait_for_completion=True,
)
print(prompt.result_text)

To deploy a custom model in DataRobot, you can follow these steps:
1. Prepare your custom model: Develop your custom model using your preferred programming language or framework.
2. Prepare your data: Ensure the data format is compatible with DataRobot and meets the necessary requirements, such as proper labeling.
3. Create a DataRobot project: Log in to your DataRobot account and create a new project. Upload your dataset and configure the project settings.
4. Train multiple models: Use DataRobot's automated machine learning capabilities to train multiple models on your dataset. This step helps you establish a baseline performance and compare it against your custom model.
5. Assess model performance: Evaluate the performance of the models trained by DataRobot. Identify the best-performing model as a benchmark to compare against your custom model.
6. Deploy custom model: Once you have established the benchmark performance, deploy your custom model using DataRobot's Deployment feature. S

In [ ]:
# Save that LLM blueprint so that you lock in the settings and can use it with
# a ComparisonPrompt or create new blueprints form it.
llm_blueprint = llm_blueprint.update(is_saved=True)
llm_blueprint

LLMBlueprint(id=6645f956dcf3d388f96a21f6, name=Overconfident DataRobot Engineer GPT 3.5, is_saved=True)

## Create vector databases

In the following cells you will create new or load existing datasets and vector databases. Then you will investigate the vector database assets.

Note: The dataset used in this accelerator is a ZIP file that contains DataRobot's publicly available English documentation. The dataset does not necessarily reflect the most recent version of the documentation, however, as the content is regularly updated. This does not impact the usefulness of the dataset for the purposes of the accelerator. The date that the content was extracted is appended to the file name.

In [ ]:
# Use a vector database with DataRobot generative AI models. Upload a dataset with DataRobot documentation.
dataset_url = "https://s3.amazonaws.com/datarobot_public_datasets/ai_accelerators/datarobot_english_documentation_5th_December.zip"
dataset = dr.Dataset.create_from_url(dataset_url)
dataset

Dataset(name='datarobot_english_documentation_5th_December.zip', id='6645f96a7701710b15f4cc18')

In [5]:
# Optionally, load an existing dataset instead
dataset = dr.Dataset.get("6645f96a7701710b15f4cc18")

In [ ]:
# Create a vector database with a set of chunking parameters
chunking_parameters = ChunkingParameters(
    embedding_model=VectorDatabaseEmbeddingModel.JINA_EMBEDDING_T_EN_V1,
    chunking_method=VectorDatabaseChunkingMethod.RECURSIVE,
    chunk_size=256,
    chunk_overlap_percentage=25,
    separators=["#", "\n\n", "\n", " ", ""],
)
vdb = VectorDatabase.create(dataset.id, chunking_parameters, use_case)
vdb

VectorDatabase(id=6645f9a3ff64531a95c4ce4b, name=Vector database for datarobot_english_documentation_5th_December.zip, execution_status=NEW)

In [4]:
# Optionally, load an existing dataset instead
vdb = VectorDatabase.get("6645f9a3ff64531a95c4ce4b")
vdb

VectorDatabase(id=6645f9a3ff64531a95c4ce4b, name=Vector database for datarobot_english_documentation_5th_December.zip, execution_status=COMPLETED)

In [ ]:
# Retrieve the vector database and make sure it has completed
assert vdb.execution_status == "COMPLETED"
vdb

VectorDatabase(id=6645f9a3ff64531a95c4ce4b, name=Vector database for datarobot_english_documentation_5th_December.zip, execution_status=COMPLETED)

In [5]:
# Download the created vector database assets
# and manually investigate whether the resulting chunks are meaningful
vdb_assets_file_name = "chunks_and_embeddings.parquet.gzip"
vdb.download_text_and_embeddings_asset(vdb_assets_file_name)
vdb_assets_df = pd.read_parquet(vdb_assets_file_name)
vdb_assets_df.head()

,text_chunks,source,page,embeddings
0,---\ntitle: Edit and use applications\ndescrip...,datarobot_english_documentation/datarobot_docs...,0,"[0.0160643, -0.061523117, -0.0508623, 0.005448..."
1,#edit-and-use-applications }\n\nWorkbench appl...,datarobot_english_documentation/datarobot_docs...,0,"[0.035402834, -0.0073612314, 0.0076815276, 0.0..."
2,See the image and table below for a brief desc...,datarobot_english_documentation/datarobot_docs...,0,"[0.036530703, -0.08172059, 4.838106e-05, 0.133..."
3,#application-folders).\n ![](images/icon-w...,datarobot_english_documentation/datarobot_docs...,0,"[0.07840027, -0.034945183, -0.020708812, 0.113..."
4,## Change themes {: #change-themes }\n\nIn the...,datarobot_english_documentation/datarobot_docs...,0,"[0.0402755, -0.091552384, 0.04012868, 0.086053..."


## Create LLM-Blueprints with vector databases (RAG)
- Copy and modify existing LLM-Blueprints
- Add vector databases to LLM-Blueprints
- Chat prompt LLM-Blueprints with vector databases (RAG)

In [ ]:
# Create a new LLM blueprint from the existing one so that you can change some
# settings and add a vector database.
new_llm_blueprint = LLMBlueprint.create_from_llm_blueprint(
    llm_blueprint, name="Copied LLM-Blueprint"
)
new_llm_blueprint

LLMBlueprint(id=6645fb33ff64531a95c4ce53, name=Copied LLM-Blueprint, is_saved=False)

In [ ]:
# Give the LLM a new personality
llm_settings["system_prompt"] = (
    "Try to answer the question only with the given context. "
    "If you don't know the answer don't make it up but instead reply with: 'I don't know'"
)
llm_settings["temperature"] = 0.0  # With the lowest degree of variability

# Add a vector database and define settings that could be different from the defaults
# when the vdb was created.
vector_database_settings = VectorDatabaseSettings(
    max_documents_retrieved_per_prompt=5,
    max_tokens=2560,
)

# Update the LLM-blueprint with the new settings including the vector database
llm_blueprint_with_vdb = new_llm_blueprint.update(
    name="GPT 3.5 + DatRobot Documentation VD",
    llm_settings=llm_settings,
    vector_database=vdb,
    # vector_database_settings=vector_database_settings,
    is_saved=True,
)

llm_blueprint_with_vdb

LLMBlueprint(id=6645fb33ff64531a95c4ce53, name=GPT 3.5 + DatRobot Documentation VD, is_saved=True)

In [ ]:
# Optionally load an existing LLMBlueprint instead
llm_blueprint_with_vdb = LLMBlueprint.get("6645fb33ff64531a95c4ce53")
llm_blueprint_with_vdb

LLMBlueprint(id=6645fb33ff64531a95c4ce53, name=GPT 3.5 + DatRobot Documentation VD, is_saved=True)

In [ ]:
# Chat with the new LLM blueprint
prompt_with_vdb = ChatPrompt.create(
    "How to deploy a custom model in datarobot?",
    llm_blueprint_with_vdb,
    wait_for_completion=True,
)
print(prompt_with_vdb.result_text)

To deploy a custom model in DataRobot, you can follow the workflow outlined below:
1. Prepare your custom model by providing the model content, which includes the compiled artifact, source code, and additional supporting files related to the model.
2. Define the model environment, which can be either a drop-in or custom environment. Drop-in environments are provided by DataRobot, while custom environments are required for specialized models and use cases.
3. Test your custom model in the Custom Model Workshop.
4. Register your custom model in the Model Registry.
5. Deploy the custom model to a centralized deployment hub, where you can monitor, manage, and govern it alongside your deployed DataRobot models.
Note: DataRobot supports custom models built in various programming languages, including Python, R, and Java.


In [ ]:
# Confirm that the LLM is using information from the vector database.
[print(citation.text) for citation in prompt_with_vdb.citations]

#deploy-a-custom-model-in-a-datarobot-environment }
Custom inference models allow you to bring your pre-trained models into DataRobot. To deploy a custom model to a DataRobot prediction environment, you can create a custom model in the Custom Model Workshop. Then, you can prepare, test, and register that model, and deploy it to a centralized deployment hub where you can monitor, manage, and govern it alongside your deployed DataRobot models. DataRobot supports custom models built in various programming languages, including Python, R, and Java.
To create and deploy a custom model in DataRobot, follow the workflow outlined below:
# Prepare custom models for deployment
Custom inference models allow you to bring your own pre-trained models to DataRobot. By uploading a model artifact to the Custom Model Workshop, you can create, test, and deploy custom inference models to a centralized deployment hub. DataRobot supports models built with a variety of coding languages, including Python, R, a

## Create external vector databases
- Load an split text datasets into documents
- Load embedding models and generate embeddings from documents
- Create FAISS indexes from embeddings
- Do similarity search on indexes

In [6]:
# Build an external vector database using the same dataset
df = dataset.get_as_dataframe()
df.head()

,document,document_file_path
0,---\ntitle: Edit and use applications\ndescrip...,datarobot_english_documentation/datarobot_docs...
1,---\ntitle: User Activity Monitor reference\nd...,datarobot_english_documentation/datarobot_docs...
2,SHAP Prediction Explanations estimate how much...,datarobot_english_documentation/datarobot_docs...
3,---\ntitle: Prepare custom models\ndescription...,datarobot_english_documentation/datarobot_docs...
4,---\ntitle: Prediction Distribution graph\ndes...,datarobot_english_documentation/datarobot_docs...


In [7]:
# Read the dataframe into langchain documents and split them using an appropriate splitter
docs = [
    Document(
        page_content=row["document"], metadata={"source": row["document_file_path"]}
    )
    for _, row in df.iterrows()
]

splitter = MarkdownTextSplitter(
    chunk_size=2000,
    chunk_overlap=1000,
)
splitted_docs = splitter.split_documents(docs)

texts = [doc.page_content for doc in splitted_docs]
metadatas = [doc.metadata for doc in splitted_docs]
print(f"Splitted docs: {len(texts)}")

Splitted docs: 5546


In [8]:
# Build a FAISS index using huggingface embeddings
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
embedding_function = SentenceTransformerEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    cache_folder="storage/deploy/sentencetransformers",
)

try:
    # Load existing db from disk if previously built
    db = FAISS.load_local("storage/deploy/faiss-db", embedding_function)
except:
    db = FAISS.from_texts(texts, embedding_function, metadatas=metadatas)
    db.save_local(folder_path="storage/deploy/faiss-db")

print(f"FAISS VectorDB has {db.index.ntotal} documents")

FAISS VectorDB has 5546 documents


In [9]:
# Test vector database retrieval / similarity search
retrieved_documents = db.similarity_search("How to deploy a custom model in datarobot?")
print(retrieved_documents)

[Document(page_content='1. Define and test model content locally (i.e., on your computer).\n\n2. Optionally, create a container environment where the model will run.\n\n3. Upload the model content and environment (if applicable) into DataRobot.\n\n## Model content {: #model-content }\n\nTo define a custom model, create a local folder containing the files listed in the table below (detailed descriptions follow the table).\n\n!!! tip\n    To ensure your assembled custom model folder has the correct contents, you can find examples of these files in the [DataRobot model template repository](https://github.com/datarobot/datarobot-user-models/tree/master/model_templates){ target=_blank } on GitHub.\n\nFile | Description | Required\n-----|-------------|---------\nModel artifact file<br>_or_<br>`custom.py`/`custom.R` file | Provide a model artifact and/or a custom code file. <ul><li>Model artifact: a serialized model artifact with a file extension corresponding to the chosen environment langua

## Deploy external vector databases
- Define custom model hooks for external vector databases
- Create and deploy external vector databases as custom models
- Make predictions on deployed external vector databases
- Validate and register external vector databases

In [ ]:
# Define hooks for deploying an unstructured custom model


def load_model(input_dir):
    """Custom model hook for loading our knowledge base."""
    import os

    from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
    from langchain.vectorstores.faiss import FAISS

    embedding_function = SentenceTransformerEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        cache_folder=f"{input_dir}/storage/deploy/sentencetransformers",
    )
    db = FAISS.load_local(f"{input_dir}/storage/deploy/faiss-db", embedding_function)
    return db


def score_unstructured(model, data, query, **kwargs) -> str:
    """Custom model hook for making completions with our vector store."""
    import json

    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import AzureChatOpenAI
    from langchain.vectorstores.base import VectorStoreRetriever

    try:
        db = model
        data_dict = json.loads(data)
        retriever = VectorStoreRetriever(vectorstore=db)
        documents = retriever.get_relevant_documents(data_dict["question"])
        relevant_text_list = [doc.page_content for doc in documents]
        rv = {"relevant": relevant_text_list}
    except Exception as e:
        rv = {"error": f"{e.__class__.__name__}: {str(e)}"}
    return json.dumps(rv), {"mimetype": "application/json", "charset": "utf8"}

In [ ]:
# Test the hooks locally
score_unstructured(
    load_model("."),
    json.dumps(
        {
            "question": "How to deploy a custom model in datarobot?",
        }
    ),
    None,
)

('{"relevant": ["1. Define and test model content locally (i.e., on your computer).\\n\\n2. Optionally, create a container environment where the model will run.\\n\\n3. Upload the model content and environment (if applicable) into DataRobot.\\n\\n## Model content {: #model-content }\\n\\nTo define a custom model, create a local folder containing the files listed in the table below (detailed descriptions follow the table).\\n\\n!!! tip\\n    To ensure your assembled custom model folder has the correct contents, you can find examples of these files in the [DataRobot model template repository](https://github.com/datarobot/datarobot-user-models/tree/master/model_templates){ target=_blank } on GitHub.\\n\\nFile | Description | Required\\n-----|-------------|---------\\nModel artifact file<br>_or_<br>`custom.py`/`custom.R` file | Provide a model artifact and/or a custom code file. <ul><li>Model artifact: a serialized model artifact with a file extension corresponding to the chosen environmen

In [ ]:
# Now let's deploy the external vector database
external_vdb_deployment = drx.deploy(
    "storage/deploy/",
    name="External DR Documentation Vector Database Deployment",
    hooks={"score_unstructured": score_unstructured, "load_model": load_model},
    extra_requirements=[
        "langchain==0.0.333",
        "faiss-cpu==1.7.4",
        "sentence-transformers==2.2.2",
    ],
    # environment_id="64c964448dd3f0c07f47d040", # Python 3.9 for app.datarobot.com
    environment_id="64d2ba178dd3f0b1fa2162f0",  # Python 3.11 for app.datarobot.com
)
# Enable storing prediction data, necessary for data export for monitoring purposes
external_vdb_deployment.dr_deployment.update_predictions_data_collection_settings(
    enabled=True
)

# Deploying custom model
  - Unable to auto-detect model type; any provided paths and files will be
    exported - dependencies should be explicitly specified using
    `extra_requirements` or `environment_id`
  - Preparing model and environment...
  - Using environment [[DataRobot] Python 3.11 GenAI
    v9](https://app.datarobot.com/model-registry/custom-environments/64d2ba178dd3f0b1fa2162f0)
    for deployment
  - Configuring and uploading custom model...
    100%|█████████████████████████████████████| 201M/201M [00:01<00:00, 156MB/s]


  - Registered custom model [External DR Documentation Vector Database
    Deployment](https://app.datarobot.com/model-registry/custom-models/66460174267e4c2ca8f82ebd/info)
    with target type: Unstructured
  - Installing additional dependencies...


  - Creating and deploying model package...


  - Created deployment [External DR Documentation Vector Database
    Deployment](https://app.datarobot.com/deployments/664602284569e66a83a14196/overview)
# Custom model deployment complete


In [ ]:
# Optionally load an already created external vector database deployment
external_vdb_deployment = drx.Deployment("664602284569e66a83a14196")

In [ ]:
# Test the deployment
external_vdb_deployment.predict_unstructured(
    {
        "question": "How to deploy a custom model in datarobot?",
        "source_documents": [],
    }
)

# Making predictions
  - Making predictions with deployment [External DR Documentation Vector
    Database
    Deployment](https://app.datarobot.com/deployments/664602284569e66a83a14196/overview)
# Predictions complete
{'relevant': ['1. Define and test model content locally (i.e., on your computer).\n\n2. Optionally, create a container environment where the model will run.\n\n3. Upload the model content and environment (if applicable) into DataRobot.\n\n## Model content {: #model-content }\n\nTo define a custom model, create a local folder containing the files listed in the table below (detailed descriptions follow the table).\n\n!!! tip\n    To ensure your assembled custom model folder has the correct contents, you can find examples of these files in the [DataRobot model template repository](https://github.com/datarobot/datarobot-user-models/tree/master/model_templates){ target=_blank } on GitHub.\n\nFile | Description | Required\n-----|-------------|---------\nModel artifact file<br>

In [ ]:
# Create a custom model vector database validation
# which is necessary to validate if an external vector database can be registered
external_vdb_validation = CustomModelVectorDatabaseValidation.create(
    prompt_column_name="question",
    target_column_name="relevant",
    deployment_id=external_vdb_deployment.dr_deployment.id,
    wait_for_completion=True,
)
external_vdb_validation

CustomModelVectorDatabaseValidation(id=66460366dcf3d388f96a221a)

In [ ]:
# Create a vector database from the validated custom model
external_vdb = VectorDatabase.create_from_custom_model(
    use_case=use_case,
    name="External DR Documentation Vector Database",
    validation_id=external_vdb_validation.id,
)
external_vdb

VectorDatabase(id=66460376dcf3d388f96a221b, name=External DR Documentation Vector Database, execution_status=COMPLETED)

## Deploy external LLMs
- Add Cohere API token to the notebook
- Define custom model hooks for Cohere API as external LLM
- Create and deploy cohere LLM API as custom models
- Make predictions on external deployed LLMs
- Validate and register external LLMs

In [ ]:
# Set the cohere credentials in DataRobot
# You can get your free cohere api token here: https://dashboard.cohere.com/api-keys
cohere_api_token = "YOUR_COHERE_API_TOKEN"
client = dr.client.get_client()
try:
    res = client.post(
        "credentials",
        json={
            "name": "cohere_api_token",
            "description": "Added from notebook",
            "credentialType": "api_token",
            "apiToken": cohere_api_token,
        },
    )
    print("Cohere credentials added to DataRobot")
except Exception:
    print("Cohere credentials already added to DataRobot")

Cohere credentials already added to DataRobot


In [15]:
# Define hooks for deploying an unstructured custom model


def load_model(*args, **kwargs):
    import cohere
    from datarobot_drum import RuntimeParameters

    cohere_api_key = RuntimeParameters.get("cohere_api_token")["apiToken"]
    return cohere.Client(cohere_api_key)


def score(data, model, **kwargs):
    import pandas as pd

    prompts = data["promptText"].tolist()
    responses = []

    for prompt in prompts:
        response = model.generate(prompt)
        responses.append(response[0].text)

    return pd.DataFrame({"responseText": responses})


# Deploy the external LLM as a custom model
external_llm_deployment = drx.deploy(
    model=None,
    name="External Cohere LLM Deployment",
    target_type="TextGeneration",
    target="responseText",
    hooks={
        "load_model": load_model,
        "score": score,
    },
    runtime_parameters=["cohere_api_token"],
    extra_requirements=["cohere==4.27", "datarobot-drum==1.10.10"],
    # environment_id="64c964448dd3f0c07f47d040", # Python 3.9 for app.datarobot.com
    environment_id="64d2ba178dd3f0b1fa2162f0",  # Python 3.11 for app.datarobot.com
)
# Enable storing prediction data, necessary for data export for monitoring purposes
external_llm_deployment.dr_deployment.update_predictions_data_collection_settings(
    enabled=True
)
external_llm_deployment.dr_deployment.id

'66549628a471c96799883414'

In [56]:
# Load an already created external LLM deployment
external_llm_deployment = drx.Deployment("66549628a471c96799883414")

In [58]:
# Do a test prediction with the external LLM deployment
prompt = "How to deploy a custom model in datarobot?"

prediction = external_llm_deployment.predict(
    pd.DataFrame(
        {
            "context": [""],
            "promptText": [prompt],
        }
    )
)
print(prediction["prediction"].values[0])

 To deploy a custom model in Datarobot, you can follow these generalized steps:

1. Train your model: Start by training your model using appropriate training data. The specific steps and tools for training depend on the type of model (e.g., machine learning, deep learning, etc.) and the framework or library being used.

2. Package your model and dependencies: Prepare your trained model and the necessary code and dependencies to run the model in a standalone environment. This may involve creating a container or package that encapsulates your model and its requirements.

3. Upload your model to Datarobot: Utilize the Datarobot platform features and tools to upload and manage your custom model. This often involves creating a new model asset in the Datarobot environment and providing the necessary files and metadata.

4. Create a new experiment: Define a new experiment that incorporates your custom model. Link your dataset to the experiment and configure any required preprocessing steps or

In [18]:
# Add a custom model LLM from a deployment. Developing the deployment is outside
# the scope of this example. This example uses a previously deployed text generation model.
# Validate it for use as an LLM.
custom_model_llm_validation = CustomModelLLMValidation.create(
    prompt_column_name="promptText",
    target_column_name="responseText",
    deployment_id=external_llm_deployment.dr_deployment.id,
    wait_for_completion=True,
)
assert custom_model_llm_validation.validation_status == "PASSED"
custom_model_llm_validation

CustomModelLLMValidation(id=665498444ca568110e7e3a7c)

## Create LLM-Blueprints from external LLMs and vector databases
- Create LLM-Blueprints from external LLMS and vector databases
- Chat prompt LLM-Blueprints build from external resources

In [19]:
# Optionally load playground and external vector database
playground = Playground.get("6645f94aff64531a95c4ce46")
external_vdb = VectorDatabase.get("66460376dcf3d388f96a221b")
custom_model_llm_validation = CustomModelLLMValidation.get("665498444ca568110e7e3a7c")

In [37]:
# Now create an LLM blueprint using the custom model LLM and the custom model vector database
external_llm_blueprint = LLMBlueprint.create(
    playground=playground,
    name="external Cohere LLM with external Faiss vdb",
    llm="custom-model",
    llm_settings={"validation_id": custom_model_llm_validation.id},
    # vector_database=external_vdb,
)
external_llm_blueprint

LLMBlueprint(id=6654a7d1f2b2bf398ae4d019, name=external Cohere LLM with external Faiss vdb, is_saved=False)

In [21]:
# Investigate the external LLMBlueprint's attributes
vars(external_llm_blueprint)

{'id': '665498874ca568110e7e3a82',
 'name': 'external Cohere LLM with external Faiss vdb',
 'description': '',
 'is_saved': False,
 'is_starred': False,
 'playground_id': '6645f94aff64531a95c4ce46',
 'llm_id': 'custom-model',
 'llm_name': 'Deployed LLM',
 'llm_settings': {'system_prompt': '',
  'validation_id': '665498444ca568110e7e3a7c',
  'external_llm_context_size': 4096},
 'creation_date': '2024-05-27T14:28:23.814111+00:00',
 'creation_user_id': '6181286e26c3fde5b5b44430',
 'last_update_date': '2024-05-27T14:28:23.814116+00:00',
 'last_update_user_id': '6181286e26c3fde5b5b44430',
 'prompt_type': 'CHAT_HISTORY_AWARE',
 'vector_database_id': '66460376dcf3d388f96a221b',
 'vector_database_settings': None,
 'vector_database_name': 'External DR Documentation Vector Database',
 'vector_database_status': 'COMPLETED',
 'vector_database_error_message': None,
 'vector_database_error_resolution': None,
 'custom_model_llm_validation_status': 'PASSED',
 'custom_model_llm_error_message': None,
 '

In [22]:
# Chat with the external LLM blueprint with an external vector database
prompt_external = ChatPrompt.create(
    "How to deploy a custom model in datarobot?",
    external_llm_blueprint,
    wait_for_completion=True,
)
print(prompt_external.result_text)

 To deploy a custom model on DataRobot, you need to define two components:

* **Model content**: This refers to the compiled artifact, source code, and supporting files for the model. The files required for this include:
  - Model artifact file or a `custom.py`/`custom.R` file - this is the trained model file or the code file with custom code implemented.
  - `model-metadata.yaml` - a file describing the model's metadata including input and output data requirements. This is used to validate the model when building and training a blueprint.
  - `requirements.txt` - a list of Python or R packages to add to the base environment.
* **Model environment**: This is the Docker image where the model will run. Environments can be default or custom environments, containing a Docker file and any necessary supporting files. 

The steps to define a custom model with these components include:

1. Define and test model content locally.
2. Optionally, create a container environment where the model will

In [ ]:
# Confirm that the LLM is using information from the vector database.
[print(citation.text) for citation in prompt_external.citations]

1. Define and test model content locally (i.e., on your computer).
2. Optionally, create a container environment where the model will run.
3. Upload the model content and environment (if applicable) into DataRobot.
## Model content {: #model-content }
To define a custom model, create a local folder containing the files listed in the table below (detailed descriptions follow the table).
!!! tip
    To ensure your assembled custom model folder has the correct contents, you can find examples of these files in the [DataRobot model template repository](https://github.com/datarobot/datarobot-user-models/tree/master/model_templates){ target=_blank } on GitHub.
File | Description | Required
-----|-------------|---------
Model artifact file<br>_or_<br>`custom.py`/`custom.R` file | Provide a model artifact and/or a custom code file. <ul><li>Model artifact: a serialized model artifact with a file extension corresponding to the chosen environment language.</li><li>Custom code: custom capabilities 

In [23]:
# Save the custom model blueprint so you can use it in a comparison
external_llm_blueprint = external_llm_blueprint.update(is_saved=True)
external_llm_blueprint

LLMBlueprint(id=665498874ca568110e7e3a82, name=external Cohere LLM with external Faiss vdb, is_saved=True)

## Comparison prompts using mulitple LLM-Blueprints
- Compare the chat prompt results of multiple LLM-Blueprints build from internal and external resources, with and without vector databases

In [ ]:
comparison_prompt = ComparisonPrompt.create(
    [llm_blueprint, llm_blueprint_with_vdb, external_llm_blueprint],
    "How to deploy a custom model in datarobot?",
    wait_for_completion=True,
)  # Compare the LLM blueprints

[print(result.result_text) for result in comparison_prompt.results]

 To deploy a custom model on DataRobot, you need to define two components:
* **Model content**: this includes the compiled artifact, source code, and additional supporting files related to the model.
* **Model environment**: this is the Docker image where the model will run. This can be a default environment provided by DataRobot, or a custom environment that you can build yourself. 
The steps to deploy your model on DataRobot are the following:
1. Define and test the model content locally on your computer.
2. (Optional) Create a container environment where the model will run.
3. Upload the model content and environment to DataRobot. 
You can define the model content by creating a local folder containing the files listed in the model content table, as described in the original text that you provided. Also, you can find examples of these files in the [DataRobot model template repository](https://github.com/datarobot/datarobot-user-models/tree/master/model_templates) on GitHub. 
When you

## Deploy LLM-Blueprints as custom model to make predictions
- Create and register custom models from LLM-Blueprints
- Deploy LLM-Blueprint custom models
- Make predictions on deployed LLM-Blueprints

In [43]:
# Optionally load an existing LLM Blueprint with external LLM and vector database
external_llm_blueprint = LLMBlueprint.get("665498874ca568110e7e3a82")
external_llm_blueprint

LLMBlueprint(id=665498874ca568110e7e3a82, name=external Cohere LLM with external Faiss vdb, is_saved=True)

In [45]:
# Create a custom model from the external LLM-Blueprint
custom_model_version = external_llm_blueprint.register_custom_model(
    prompt_column_name="promptText",
    target_column_name="responseText",
)
(custom_model_version, custom_model_version.id)

(CustomModelVersion('v1.0'), '6654af244572240ea3cd11bb')

In [49]:
# Optionally load the last custom model version from a custom model ID
custom_model_version = dr.CustomModelVersion.list("6654af23828abe299dcd1318")[0]
(custom_model_version, custom_model_version.id)

(CustomModelVersion('v2.1'), '665584c36c8386e31e82a080')

In [50]:
# Deloy the custom model version
deployment = dr.Deployment.create_from_custom_model_version(
    custom_model_version_id=custom_model_version.id,
    label=f"Deployment from custom model with external vector database and LLM",
    default_prediction_server_id=dr.PredictionServer.list()[0].id,
    max_wait=1200,
)
print(deployment, deployment.id)

Deployment(Deployment from custom model with external vector database and LLM) 665585998b9b7fecc34e7f31


In [51]:
# Cast it as a drx deployment because it has a more convenient predict method
deployment = drx.Deployment("665585998b9b7fecc34e7f31")
deployment.dr_deployment

Deployment(Deployment from custom model with external vector database and LLM)

In [54]:
# Make predictions with the deployed LLM-Blueprint
# using our external Cohere LLM and our external FAISS index deployment
prompt = "How to deploy a custom model in datarobot?"

prediction = deployment.predict(
    pd.DataFrame(
        {
            "context": [""],
            "promptText": [prompt],
        }
    )
)
print(prediction["prediction"].values[0])

 To deploy a custom model in DataRobot, you need to define and test the model content locally on your computer, create a container environment where the model will run, and upload the model content and environment into DataRobot. These are the general steps to follow, along with some considerations:

1. **Define and test model content locally**: Create a local folder containing the required files for the custom model, including the model artifact file or a custom code file (e.g., `custom.py` or `custom.R`), a `model-metadata.yaml` file describing the model's metadata, and optionally, a `requirements.txt` file with a list of required Python or R packages. You can find examples of these files in the DataRobot model template repository on GitHub.

2. **Create a container environment (optional)**: If the model requires a specific environment, you may need to create a Docker container with the required libraries and dependencies. You can use the provided Dockerfile and supporting files to d

## Conclusion

This acclerator showcases the capabilities of DataRobot's new generative AI capabilities, including LLM playgrounds and vector databases essential for Retriever Augmented Generation (RAG) workflows. It provided a comprehensive guide on creating and using DataRobot generative AI playgrounds, LLM-Blueprints, and vector databases. The accelerator demonstrated how to deploy and integrate both internal and external vector databases and LLMs, including the use of external resources like the Cohere API. Additionally, it covered the comparative analysis of multiple LLM-Blueprints and the deployment of these blueprints for practical applications and predictions.